In [10]:
import os
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import glob
import io
from PIL import Image
import tensorflow as tf
from tqdm import tqdm

In [2]:
LABEL_PATH = r"C:\Users\parzi\OneDrive - Tribhuvan University\Desktop\Minor Project\Monument Detection with CNN\Monument Object Detection\helper\Report\final.csv"
ignore_list = ['kotilingeshvara', 'til mahadev narayan temple', 'king statue']
mistake_list = ['degutale temple', 'kritipur tower']
correct_list = ['degu tale temple_KDS', 'kirtipur tower', 'degu tale']

In [7]:
POSSIBLE_LABELS = dict()
def csv_to_label_map_dict(path):
    df = pd.read_csv(path)
    class_list = list(df['class'])
    final_class_list = list()
    for class_name in class_list:
        if class_name not in ignore_list and class_name not in correct_list:
            if class_name in mistake_list:
                if class_name == 'degutale temple' or class_name == 'degu tale':
                    class_name = correct_list[0]
                elif class_name == 'kritipur tower':
                    class_name = correct_list[1]
            final_class_list.append(class_name)
    class_index = 1
    res_dict = {}
    for class_label in final_class_list:
        res_dict[class_label] = class_index
        class_index += 1
    return res_dict

POSSIBLE_LABELS = csv_to_label_map_dict(LABEL_PATH)
POSSIBLE_LABELS['bg'] = 0

In [4]:
def check_class_validity(class_name):
    if class_name == 'degutale temple' or class_name == 'degu tale':
        return 'degu tale temple_KDS'
    elif class_name == 'kritipur tower':
        return 'kirtipur tower'
    elif class_name not in list(POSSIBLE_LABELS.keys()):
        return None
    else:
        return class_name

In [11]:
class_list = []
def xml_to_csv(path):
    xml_list = []
    for xml_file in tqdm(glob.glob(path + '/*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        all_objects = root.findall('object')
        changed_filename = xml_file.split('\\')[-1].split('.')[0] + '.JPG'
        # Handling xml parser for random background images
        if len(all_objects) != 0:
            for member in all_objects:
                class_label = check_class_validity(member[0].text)
                if class_label is not None:
                    value = (changed_filename,
                             int(root.find('size')[0].text),
                             int(root.find('size')[1].text),
                             class_label,
                             int(member[4][0].text),
                             int(member[4][1].text),
                             int(member[4][2].text),
                             int(member[4][3].text)
                             )
                    if class_label not in class_list:
                        class_list.append(class_label)
                    xml_list.append(value)
        else:
            # Setting background class to 'bg', and four bbox coordinates all to neg 1.
            value = (changed_filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     'bg',
                      -1,
                      -1,
                      -1,
                      -1)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [26]:
df = xml_to_csv(r"F:\Minor Data Collection\Final Image Data\Monument Original 512\Augmented 512 v2 (PDS included)\All Splitted\train\Annotations")

100%|██████████████████████████████████████████████████████████████████████████| 19609/19609 [00:03<00:00, 6432.66it/s]


In [27]:
new_df = df.groupby(df['class']).count()
new_df = new_df[['filename']]

In [29]:
new_df.to_excel('train_augmented_dataset.xlsx')